In [1]:
import numpy as np
import webbrowser
from aocd.models import Puzzle
from scipy.signal import correlate2d
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import re
%matplotlib inline
puzzle = Puzzle(year=2021, day=23)

In [2]:
webbrowser.open(puzzle.url);

## Part 1

In [632]:
test_input_data = """
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
""".strip()

In [616]:
neighbours = {
    15: [(16, 1)],
    16: [(15, 1), (18, 2), (31, 2)],
    18: [(16, 2), (31, 2), (33, 2), (20, 2)],
    20: [(18, 2), (22, 2), (33, 2), (35, 2)],
    22: [(20, 2), (24, 2), (35, 2), (37, 2)],
    24: [(22, 2), (25, 1), (37, 2)],
    25: [(24, 1)],
    31: [(16, 2), (18, 2), (45, 1)],
    33: [(18, 2), (20, 2), (47, 1)],
    35: [(20, 2), (22, 2), (49, 1)],
    37: [(22, 2), (24, 2), (51, 1)],
    45: [(31, 1)],
    47: [(33, 1)],
    49: [(35, 1)],
    51: [(37, 1)],
}

rooms = {
    "hallway": {15, 16, 18, 20, 22, 24, 25},
    "A": {31, 45},
    "B": {33, 47},
    "C": {35, 49},
    "D": {37, 51}
}

energy = {"A": 1, "B": 10, "C": 100, "D": 1000}

In [617]:
paths = {}
def generate_path(path, total_dist):
    if len(path) >= 2:
        path_key = (path[0], path[-1])
        if path_key in paths:  # already have a path, check if the new one is shorter
            curr_path, curr_dist = paths[path_key]
            if total_dist < curr_dist:
                paths[(path[0], path[-1])] = path[:], total_dist
        else:  # a new path
            paths[(path[0], path[-1])] = path[:], total_dist

    for neigh, dist in neighbours[path[-1]]:
        if neigh in path:  # already part of the path
            continue
        path.append(neigh)
        generate_path(path, total_dist + dist)
        path.pop()

for room in neighbours:
    generate_path([room], 0)

In [618]:
def find(s, ch):
    return [i for i, c in enumerate(s) if c == ch]

In [633]:
def initial_positions(input_data):
    positions = {}
    for ch in ["A", "B", "C", "D"]:
        for i in find(input_data, ch):
            positions[i] = ch
    return positions

In [634]:
def is_blocking(positions, room_ch, pos):
    min_pos, max_pos = min(rooms[room_ch]), max(rooms[room_ch])
    return pos == min_pos and positions.get(max_pos) != room_ch

In [635]:
def is_room_free(positions, room_ch):
    for pos in rooms[room_ch]:
        if positions.get(pos, room_ch) != room_ch:
            return False
    return True

In [636]:
def is_path_free(positions, path):
    for pos in path[1:]:
        if pos in positions:
            return False
    return True

In [637]:
def is_finished(positions):
    for pos, ch in positions.items():
        if pos not in rooms[ch]:
            return False
    return True

In [638]:
import heapq

class MinHeap:
    def __init__(self, key, data=()):
        self.key = key
        self.heap = [(self.key(d), d) for d in data]
        heapq.heapify(self.heap)

    def push(self, item):
        decorated = self.key(item), item
        heapq.heappush(self.heap, decorated)

    def pop(self):
        decorated = heapq.heappop(self.heap)
        return decorated[1]

    def pushpop(self, item):
        decorated = self.key(item), item
        dd = heapq.heappushpop(self.heap, decorated)
        return dd[1]

    def replace(self, item):
        decorated = self.key(item), item
        dd = heapq.heapreplace(self.heap, decorated)
        return dd[1]

    def __len__(self):
        return len(self.heap)

In [639]:
def solve(initial_positions):
    heap = MinHeap(lambda p: p[0])
    heap.push((0, tuple(initial_positions.items())))
    seen = set()

    while True:
        curr_cost, positions = heap.pop()
        if positions in seen:
            continue  # we had that already, lets avoid loops
        seen.add(positions)
        positions = dict(positions)
        if is_finished(positions):
            return curr_cost

        for pos, ch in tuple(positions.items()):
            if pos in rooms[ch] and not is_blocking(positions, ch, pos):  # we are already in our target location
                continue
            if pos in rooms["hallway"]:  # we are in the hallway
                # from the hallway we can only move in our target room, but only if it is free
                if not is_room_free(positions, ch):  # no valid move here
                    continue
                room_pos = max(rooms[ch])
                if room_pos in positions:  # lower spot already occupied
                    room_pos = min(rooms[ch])
                path, dist = paths[pos, room_pos]
                if is_path_free(positions, path):
                    new_pos = dict(positions)
                    new_pos[room_pos] = ch
                    del new_pos[pos]
                    heap.push((curr_cost + dist * energy[ch], tuple(new_pos.items())))
            else:  # we are in a room
                for hallway_pos in rooms["hallway"]:  # we can move somewhere to the hallway
                    path, dist = paths[pos, hallway_pos]
                    if is_path_free(positions, path):
                        new_pos = dict(positions)
                        new_pos[hallway_pos] = ch
                        del new_pos[pos]
                        heap.push((curr_cost + dist * energy[ch], tuple(new_pos.items())))


In [630]:
result = solve(initial_positions(test_input_data))
result

12521

In [641]:
%%time
result = solve(initial_positions(puzzle.input_data))
result

CPU times: user 1min 21s, sys: 958 ms, total: 1min 22s
Wall time: 1min 22s


19059

In [544]:
puzzle.answer_a = result

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


## Part 2

In [629]:
def augment_input(input_data):
    lines = input_data.split("\n")
    lines.insert(3, "  #D#C#B#A#")
    lines.insert(4, "  #D#B#A#C#")
    return "\n".join(lines)

In [658]:
neighbours = {
    15: [(16, 1)],
    16: [(15, 1), (18, 2), (31, 2)],
    18: [(16, 2), (31, 2), (33, 2), (20, 2)],
    20: [(18, 2), (22, 2), (33, 2), (35, 2)],
    22: [(20, 2), (24, 2), (35, 2), (37, 2)],
    24: [(22, 2), (25, 1), (37, 2)],
    25: [(24, 1)],
    31: [(16, 2), (18, 2), (45, 1)],
    33: [(18, 2), (20, 2), (47, 1)],
    35: [(20, 2), (22, 2), (49, 1)],
    37: [(22, 2), (24, 2), (51, 1)],
    45: [(31, 1), (57, 1)],
    47: [(33, 1), (59, 1)],
    49: [(35, 1), (61, 1)],
    51: [(37, 1), (63, 1)],
    57: [(45, 1), (69, 1)],
    59: [(47, 1), (71, 1)],
    61: [(49, 1), (73, 1)],
    63: [(51, 1), (75, 1)],
    69: [(57, 1)],
    71: [(59, 1)],
    73: [(61, 1)],
    75: [(63, 1)],
}

rooms = {
    "hallway": {15, 16, 18, 20, 22, 24, 25},
    "A": {31, 45, 57, 69},
    "B": {33, 47, 59, 71},
    "C": {35, 49, 61, 73},
    "D": {37, 51, 63, 75}
}

energy = {"A": 1, "B": 10, "C": 100, "D": 1000}

In [659]:
paths = {}
def generate_path(path, total_dist):
    if len(path) >= 2:
        path_key = (path[0], path[-1])
        if path_key in paths:  # already have a path, check if the new one is shorter
            curr_path, curr_dist = paths[path_key]
            if total_dist < curr_dist:
                paths[(path[0], path[-1])] = path[:], total_dist
        else:  # a new path
            paths[(path[0], path[-1])] = path[:], total_dist

    for neigh, dist in neighbours[path[-1]]:
        if neigh in path:  # already part of the path
            continue
        path.append(neigh)
        generate_path(path, total_dist + dist)
        path.pop()

for room in neighbours:
    generate_path([room], 0)

In [661]:
def is_blocking(positions, room_ch, pos):
    min_pos, max_pos = min(rooms[room_ch]), max(rooms[room_ch])
    for room_pos in rooms[room_ch]:
        if room_pos > pos and positions.get(room_pos) != room_ch:
            return True
    return False

In [662]:
def is_room_free(positions, room_ch):
    for pos in rooms[room_ch]:
        if positions.get(pos, room_ch) != room_ch:
            return False
    return True

In [691]:
def get_furthest_free_room_pos(positions, room_ch):
    for pos in sorted(rooms[room_ch], reverse=True):
        if pos not in positions:
            return pos

In [663]:
def is_path_free(positions, path):
    for pos in path[1:]:
        if pos in positions:
            return False
    return True

In [664]:
def is_finished(positions):
    for pos, ch in positions.items():
        if pos not in rooms[ch]:
            return False
    return True

In [703]:
def solve(initial_positions):
    heap = MinHeap(lambda p: p[0])
    heap.push((0, tuple(initial_positions.items())))
    seen = set()

    while True:
        curr_cost, positions = heap.pop()
        if len(heap) == 0:
            print(curr_cost, positions)
        if positions in seen:
            continue  # we had that already, lets avoid loops
        seen.add(positions)
        positions = dict(positions)
        if is_finished(positions):
            return curr_cost

        for pos, ch in tuple(positions.items()):
            if pos in rooms[ch] and not is_blocking(positions, ch, pos):  # we are already in our target location
                continue
            if pos in rooms["hallway"]:  # we are in the hallway
                # from the hallway we can only move in our target room, but only if it is free
                if not is_room_free(positions, ch):  # no valid move here
                    continue
                room_pos = get_furthest_free_room_pos(positions, ch)
                path, dist = paths[pos, room_pos]
                if is_path_free(positions, path):
                    new_pos = dict(positions)
                    new_pos[room_pos] = ch
                    del new_pos[pos]
                    heap.push((curr_cost + dist * energy[ch], tuple(new_pos.items())))
            else:  # we are in a room
                for hallway_pos in rooms["hallway"]:  # we can move somewhere to the hallway
                    path, dist = paths[pos, hallway_pos]
                    if is_path_free(positions, path):
                        new_pos = dict(positions)
                        new_pos[hallway_pos] = ch
                        del new_pos[pos]
                        heap.push((curr_cost + dist * energy[ch], tuple(new_pos.items())))


In [687]:
def visualize(positions):
    template = """
#############
#...........#
###.#.#.#.###
  #.#.#.#.#
  #.#.#.#.#
  #.#.#.#.#
  #########
    """.strip()
    out = ""
    for i, ch in enumerate(template):
        if i in positions:
            out += positions[i]
        else:
            out += ch
    return out

In [704]:
%%time
result = solve(initial_positions(augment_input(test_input_data)))
result

0 ((51, 'A'), (61, 'A'), (69, 'A'), (75, 'A'), (31, 'B'), (35, 'B'), (49, 'B'), (59, 'B'), (33, 'C'), (47, 'C'), (63, 'C'), (73, 'C'), (37, 'D'), (45, 'D'), (57, 'D'), (71, 'D'))
CPU times: user 2min 2s, sys: 1.31 s, total: 2min 4s
Wall time: 2min 4s


44169

In [705]:
%%time
result = solve(initial_positions(augment_input(puzzle.input_data)))
result

0 ((51, 'A'), (61, 'A'), (71, 'A'), (73, 'A'), (35, 'B'), (49, 'B'), (59, 'B'), (75, 'B'), (33, 'C'), (37, 'C'), (47, 'C'), (63, 'C'), (31, 'D'), (45, 'D'), (57, 'D'), (69, 'D'))
CPU times: user 1min 37s, sys: 1.25 s, total: 1min 39s
Wall time: 1min 39s


48541

In [706]:
puzzle.answer_b = result

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 23! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
